In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
drive='/content/gdrive/My Drive/python/fever/'  #Change the path to the correct directory where the cleaned_data.csv file is present
os.chdir(drive)
print(os.getcwd())

/content/gdrive/My Drive/python/fever


In [ ]:
#Download the official scoring. No need to run everytime as the file is persisted in the google drive
!wget https://raw.githubusercontent.com/sheffieldnlp/fever-scorer/master/src/fever/scorer.py

--2020-11-29 19:33:17--  https://raw.githubusercontent.com/sheffieldnlp/fever-scorer/master/src/fever/scorer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6298 (6.2K) [text/plain]
Saving to: ‘scorer.py’

scorer.py           100%[===================>]   6.15K  --.-KB/s    in 0.001s  

2020-11-29 19:33:18 (4.20 MB/s) - ‘scorer.py’ saved [6298/6298]



In [ ]:
!ls

baseline_classifier_5000_model.h5
baseline_classifier_inferbert_model.h5
baseline_classifier_infer_roberta_model.h5
body_index
cache_dir
combined_replaced_pronouns.jsonl
fever_bert_base_encased
fever_retrieval_distil_roberta_base
fever_retrieval_roberta_base
fever_roberta_base
fever_roberta_base_concat_evidence
formatted_data_train_3_class.jsonl
formatted_data_train_concat_2_class.jsonl
license.html
__MACOSX
neural_network.py
pages.txt
__pycache__
retrieval_data_2_class.jsonl
runs
sample_data.json
scorer.py
tfidf_vectorizer_5000_features.pickle
title_index
wiki-pages
wiki-pages.zip


In [ ]:
!pip install simpletransformers==0.49.0

     |████████████████████████████████| 225kB 13.2MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 1.8MB 26.1MB/s 
     |████████████████████████████████| 317kB 39.9MB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
     |████████████████████████████████| 1.3MB 44.1MB/s 
     |████████████████████████████████| 7.4MB 48.0MB/s 
     |████████████████████████████████| 2.9MB 47.2MB/s 
     |████████████████████████████████| 163kB 56.0MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
     |████████████████████████████████| 133kB 55.3MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
     |████████████████████████████████| 890kB 60.0MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
     |████████████████████████████████| 133kB 52.9MB/s 
     |████████████████████████████████| 112kB 52.1MB/s 
     |████████████████████████████████| 4.5MB 49.3MB/s 
     |████████████████████████████████| 6.9MB 43.6MB/s

In [ ]:
!pip uninstall -y pandas
!pip install pandas==0.25.1

Uninstalling pandas-1.1.4:
  Successfully uninstalled pandas-1.1.4
     |████████████████████████████████| 10.5MB 11.5MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 0.25.1 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 0.25.1 which is incompatible.


In [ ]:
import json, nltk
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pd.__version__

'0.25.1'

In [ ]:
INPUT_FILE = 'sample_data.json'
df=pd.read_json(INPUT_FILE)

In [ ]:
df

,id,verifiable,label,claim,evidence,pages
0,91198,NOT VERIFIABLE,NOT ENOUGH INFO,Colin Kaepernick became a starting quarterback...,"[[[108548, None, None, None]]]",{}
1,194462,NOT VERIFIABLE,NOT ENOUGH INFO,Tilda Swinton is a vegan.,"[[[227768, None, None, None]]]",{}
2,137334,VERIFIABLE,SUPPORTS,Fox 2000 Pictures released the film Soul Food.,"[[[289914, 283015, Soul_Food_-LRB-film-RRB-, 0...",{'Soul_Food_-LRB-film-RRB-': {'code': '091-406...
3,166626,NOT VERIFIABLE,NOT ENOUGH INFO,Anne Rice was born in New Jersey.,"[[[191656, None, None, None], [191657, None, N...",{}
4,111897,VERIFIABLE,REFUTES,Telemundo is a English-language television net...,"[[[131371, 146144, Telemundo, 0]], [[131371, 1...","{'Noticiero_Telemundo': {'code': '073-19441', ..."
...,...,...,...,...,...,...
95,186996,VERIFIABLE,SUPPORTS,Bermuda Triangle is also known by another name.,"[[[218184, 224536, Bermuda_Triangle, 0]]]","{'Bermuda_Triangle': {'code': '015-40174', 'se..."
96,102702,NOT VERIFIABLE,NOT ENOUGH INFO,Ashton Kutcher was directed by George Clooney.,"[[[120574, None, None, None]]]",{}
97,145446,VERIFIABLE,SUPPORTS,Topman has clothing outlets in six Irish citie...,"[[[168755, 182455, Topman, 1]]]","{'Topman': {'code': '1', 'sent': ['0 Topman is..."
98,68084,VERIFIABLE,SUPPORTS,Wales' population changed.,"[[[84694, 96909, Wales, 16]]]","{'GO_Wales': {'code': '036-25635', 'sent': ['0..."


In [ ]:
df['claim'].describe()

count                                              100
unique                                             100
top       Murda Beatz's real name is Marshall Mathers.
freq                                                 1
Name: claim, dtype: object

In [ ]:
pages_map={}
for(claim, pages) in zip(df['claim'], df['pages']):
  #print("Claim={} pages={}".format(claim, pages))
  pages_map[claim] = pages
#print(pages_map)


In [ ]:
def top_n_sentences(claim, articles, n =5, val = 1):
    if val == 1:
        vect = TfidfVectorizer(min_df=1, stop_words="english")
        corpus = []
        line_corpus = []
        corpus.append(claim)
        for key in articles:
            for line in articles[key]['sent']:
                temp = line.split()
                if len(temp) <= 3:
                    continue
                corpus.append(' '.join(temp[1:]))
                line_corpus.append((key, int(temp[0]), ' '.join(temp[1:])))
        tfidf = vect.fit_transform(corpus)                                                                                                                                                                                                                       
        # pairwise_similarity = tfidf[0] * tfidf.T
        pairwise_similarity = cosine_similarity(tfidf[0], tfidf)
        arr = pairwise_similarity[0]
        indices = arr.argsort()[-(n+1):][::-1][1:]
        res = []
        for i in indices:
            res.append(line_corpus[i-1])
        return res

In [ ]:
# claim = 'Savages was exclusively a German film.'
top_n_sentences(df.claim[2], df.pages[2], n=10)

[('Soul_Food_-LRB-film-RRB-',
  0,
  "Soul Food is a 1997 American comedy-drama film produced by Kenneth `` Babyface '' Edmonds , Tracey Edmonds and Robert Teitel and released by Fox 2000 Pictures . Robert Teitel Robert Teitel comedy-drama film comedy-drama film Tracey Edmonds Tracey Edmonds Fox 2000 Pictures Fox 2000 Pictures"),
 ('Soul_Food_-LRB-soundtrack-RRB-',
  0,
  'Soul Food is the soundtrack to the 1997 film of the same name . 1997 film of the same name Soul Food (film)'),
 ('Fox_Searchlight_Pictures',
  0,
  "Fox Searchlight Pictures is an American film distribution company within the Fox Entertainment Group , a sister company of the larger Fox studio 20th Century Fox , all owned by Rupert Murdoch 's 21st Century Fox . 21st Century Fox 21st Century Fox 20th Century Fox 20th Century Fox Fox Entertainment Group Fox Entertainment Group Rupert Murdoch Rupert Murdoch"),
 ('Soul_Food', 0, 'Soul food is a type of cuisine . Soul food Soul food'),
 ('Soul_Food',
  5,
  'Soul Food -LRB

In [ ]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging
model = ClassificationModel("bert", "fever_bert_base_encased", num_labels=3, use_cuda=False)

In [ ]:
model = ClassificationModel("roberta", "fever_roberta_base", num_labels=3, use_cuda=False)

In [ ]:
def predict(claim):
  articles=[]
  sentence_ids=[]
  sentences=[]
  obj={}
  label_dict = {"SUPPORTS":0, "REFUTES": 1, "NOT ENOUGH INFO": 2}
  predicted_label = "NOT ENOUGH INFO"
  predicted_evidence = [["None", 0]]
  
  #print("Claim={}".format(claim))
  possible_evidences = top_n_sentences(claim, pages_map[claim], n=5, val=1)
  #print(possible_evidences)
  if len(possible_evidences)==0:
    #Not enough info
    obj["predicted_label"] = predicted_label
    obj["predicted_evidence"] = predicted_evidence
    return obj
  
  for (article, sentence_id, sentence) in possible_evidences:
    #print(article, sentence_id, sentence)
    articles.append(article)
    sentence_ids.append(sentence_id)
    sentences.append(sentence)
  
  claims = [claim]*len(sentences)
  evidences = sentences
  X_test = [[s1, s2] for s1, s2 in zip(claims, evidences)]
  #print(X_test[:5])
  predictions, raw_outputs = model.predict(X_test)
  #print(predictions)
  num_supports = predictions.count(0)
  num_refutes = predictions.count(1)
  print("Support={} refutes={}".format(num_supports, num_refutes))
  evidences = []
  if num_supports > 0:
    predicted_label = "SUPPORTS"
    for (article, sentence_id, prediction) in zip(articles, sentence_ids, predictions):
      if prediction == 0:
        evidence = [article, sentence_id]
        evidences.append(evidence)
  elif num_refutes > 0:
    predicted_label = "REFUTES"
    for (article, sentence_id, prediction) in zip(articles, sentence_ids, predictions):
      if prediction == 1:
        evidence = [str(article), int(sentence_id)]
        evidences.append(evidence)
  obj["predicted_label"] = predicted_label
  obj["predicted_evidence"] = evidences
  return obj

In [ ]:
#Predictions with BERT
from scorer import fever_score
#Start evaluating the claims
predictions=[]
actuals=[]
howmany=100
for claim, label, evidence in zip(df.claim[:howmany], df.label[:howmany], df.evidence[:howmany]):
  pred = predict(claim)
  #print("Prediction=", pred)
  predictions.append(pred)
  actual={}
  actual['label']=label
  actual['evidence'] = evidence
  actuals.append(actual)
strict_score, label_accuracy, precision, recall, f1 = fever_score(predictions, actuals)
print("strict_score={} label_acccuracy={} precision={} recall={} f1={}".format(strict_score, label_accuracy, precision, recall, f1))


Support=1 refutes=1



Support=5 refutes=0



Support=1 refutes=2



Support=5 refutes=0



Support=0 refutes=5


...


Support=2 refutes=1



Support=4 refutes=0



Support=4 refutes=0



Support=1 refutes=0



Support=1 refutes=0
strict_score=0.49 label_acccuracy=0.72 precision=0.31396396396396387 recall=0.3918918918918919 f1=0.3486262267372669


In [ ]:
#Predictions with RoBERTa
from scorer import fever_score
#Start evaluating the claims
predictions=[]
actuals=[]
howmany=100
for claim, label, evidence in zip(df.claim[:howmany], df.label[:howmany], df.evidence[:howmany]):
  pred = predict(claim)
  #print("Prediction=", pred)
  predictions.append(pred)
  actual={}
  actual['label']=label
  actual['evidence'] = evidence
  actuals.append(actual)
strict_score, label_accuracy, precision, recall, f1 = fever_score(predictions, actuals)
print("strict_score={} label_acccuracy={} precision={} recall={} f1={}".format(strict_score, label_accuracy, precision, recall, f1))


Support=4 refutes=1



Support=2 refutes=3



Support=1 refutes=3



Support=5 refutes=0



Support=0 refutes=5


...


Support=4 refutes=1



Support=4 refutes=0



Support=5 refutes=0



Support=5 refutes=0



Support=0 refutes=5
strict_score=0.53 label_acccuracy=0.8 precision=0.1405405405405405 recall=0.40540540540540543 f1=0.20872357506020867


In [ ]:
def jaccard_similarity(claim, evidence):
    claim_set = set(claim.split())
    evidence_set = set(evidence.split())
    #print(claim_set)
    #print(evidence_set)
    overlap = 1.0*len(claim_set.intersection(evidence_set))/len(claim_set)
    return overlap

In [ ]:
model.load_weights("baseline_classifier_5000_model.h5")


In [ ]:
def vector_to_label(vector):
    vector = vector.tolist()
    return vector.index(max(vector))

In [ ]:
#Predicting with TF-IDF model
claim = 'Neal Schon was named in 1954';
article_ids, sentence_ids, sentences = make_features(claim)
print(sentences)
claims = [claim]*len(sentences)
print(claims)
evidences = sentences
claims_features = vectorizer.transform(claims)
evidence_features = vectorizer.transform(evidences)

print("Shape of claims=", claims_features.shape)
print("Shape of evidences=", evidence_features.shape)
print("Converting claims to dense")
claims_features = claims_features.todense()
print("Converting evidence to dense")
evidence_features = evidence_features.todense()
dot_products = []
for (c, e) in zip(claims_features, evidence_features):
    dp = np.dot(c, e.T)
    dot_products.append(dp)
dot_products = np.array(dot_products)
print("Shape of dot_products=", dot_products.shape)
dot_products = dot_products.reshape(evidence_features.shape[0], -1)
print("AFter reshaping Shape of dot_products=", dot_products.shape)
all_features = np.concatenate((claims_features, dot_products, evidence_features), axis=1)
print("Shape of all=", all_features.shape)
y_pred = model.predict(all_features)
y_pred = [vector_to_label(vector) for vector in y_pred]
print(y_pred)
label_dict = {"SUPPORTS":0, "REFUTES": 1, "NOT ENOUGH INFO": 2}
pred_to_labels = dict(map(reversed, label_dict.items())) 
for (evidence, prediction) in zip(evidences, y_pred):
  pred_label = pred_to_labels[prediction]
  print(evidence, pred_label)

Proper nouns ['Neal', 'Schon']
Searching database
109406685
134226865
677015653
773409603
Title=Neal_Schon, #sentences=8
Title=Heinrich_Neal, #sentences=35
Title=Max_Neal, #sentences=20
Title=Neal_Dow_House, #sentences=6
Title=Nasir_Schon, #sentences=12
['Neal Joseph Schon -LRB- born February 27 , 1954 -RRB- is an American rock guitarist , songwriter , and vocalist , best known for his work with the bands Journey and Bad English . Journey Journey (band) Bad English Bad English rock rock music', "He is Journey 's last original constant member , having participated in every album and tour to date . Journey Journey (band)", 'He was a member of the rock band Santana before forming Journey , and was also an original member of Hardline . Santana Santana (band) Journey Journey (band) Hardline Hardline (band) rock rock music', 'Schon was inducted into the Oklahoma Music Hall of Fame on August 23 , 2013 . Oklahoma Oklahoma Oklahoma Music Hall of Fame Oklahoma Music Hall of Fame', 'Schon was ind

In [ ]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging

In [ ]:
model = ClassificationModel("bert", "fever_bert_base_encased", num_labels=3, use_cuda=False)

In [ ]:
#Predicting with tansformers model
claim = "Tetris has sold millions of physical copies."
article_ids, sentence_ids, sentences = make_features(claim)
print(sentences)

claims = [claim]*len(sentences)
print(claims)
evidences = sentences
X_test = [[s1, s2] for s1, s2 in zip(claims, evidences)]
print(X_test[:5])
predictions, raw_outputs = model.predict(X_test)
print(predictions)
label_dict = {"SUPPORTS":0, "REFUTES": 1, "NOT ENOUGH INFO": 2}
pred_to_labels = dict(map(reversed, label_dict.items()))
for (claim, evidence, prediction.) in zip(claims, evidences, predictions):
  js = jaccard_similarity(claim, evidence)
  pred_label = pred_to_labels[prediction]
  print("JS=", js, pred_label, evidence, pred_label)

Proper nouns ['Tetris']
Searching database
148207698
827907372
Title=Ecstasy_of_Order-COLON-_The_Tetris_Masters, #sentences=8
Title=Tetris_Ultimate, #sentences=3
Title=Tetris_Splash, #sentences=7
Title=Tetris_effect, #sentences=9
Title=The_New_Tetris, #sentences=9
Title=Tetris_2_-LRB-1990_video_game-RRB-, #sentences=4
Title=Tetris_Online,_Inc., #sentences=5
Title=Tetris_-LRB-Atari-RRB-, #sentences=16
Title=Tetris_Battle_Gaiden, #sentences=3
Title=List_of_Tetris_variants, #sentences=3
Title=Tetris_Dekaris, #sentences=10
Title=Pokémon_Tetris, #sentences=3
Title=Tetris_Friends, #sentences=14
Title=The_Tetris_Company, #sentences=3
Title=Tetris_Blitz, #sentences=5
Title=Tetris_Worlds, #sentences=10
Title=Tetris_2_-LRB-1993_video_game-RRB-, #sentences=2
Title=Puyo_Puyo_Tetris, #sentences=13
Title=Tetris_Attack, #sentences=12
Title=Tetris_-LRB-Game_Boy-RRB-, #sentences=7
['Ecstasy of Order : The Tetris Masters is a 2011 American documentary film that follows the lives of several gamers from a


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2]
JS= 0.2857142857142857 NOT ENOUGH INFO Ecstasy of Order : The Tetris Masters is a 2011 American documentary film that follows the lives of several gamers from around the country as they prepare to compete in the 2010 Classic Tetris World Championship held in Los Angeles , California . documentary film documentary film Classic Tetris World Championship Classic Tetris World Championship Tetris Tetris NOT ENOUGH INFO
JS= 0.42857142857142855 NOT ENOUGH INFO It recounts the development and rise of Tetris as one of the most-played video games of all-time , the role it has played in shaping the lives of the gamers it chronicles , the mystery surrounding the whereabouts of former Nintendo World Champion Th